In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, cv2, random, time, shutil, csv
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
np.random.seed(42)
%matplotlib inline 

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D, Lambda, Dropout, InputLayer, Input
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img
from keras import Sequential
from keras.layers import Dense, Dropout, InputLayer, Lambda, Input

Using TensorFlow backend.


In [2]:
#Data Paths
train_dir = '/kaggle/input/dog-breed-identification/train'
test_dir = '/kaggle/input/dog-breed-identification/test'

In [3]:
#Read train labels.
labels_dataframe = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
#Read sample_submission file to be modified by pridected labels.
sample_df = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv')
#Incpect labels_dataframe.
labels_dataframe.head(5)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [4]:
#Create list of alphabetically sorted labels.
dog_breeds = sorted(list(set(labels_dataframe['breed'])))
n_classes = len(dog_breeds)

In [5]:
#Map each label string to an integer label.
class_to_num = dict(zip(dog_breeds, range(n_classes)))

In [6]:
def images_to_array(data_dir, labels_dataframe, img_size = (224,224,3)):

    images_names = labels_dataframe['id']
    images_labels = labels_dataframe['breed']
    data_size = len(images_names)
    #initailize output arrays.
    X = np.zeros([data_size, img_size[0], img_size[1], img_size[2]], dtype=np.uint8)
    y = np.zeros([data_size,1], dtype=np.uint8)
    #read data and lables.
    for i in tqdm(range(data_size)):
        image_name = images_names[i]
        img_dir = os.path.join(data_dir, image_name+'.jpg')
        img_pixels = load_img(img_dir, target_size=img_size)
        X[i] = img_pixels
        
        image_breed = images_labels[i]
        y[i] = class_to_num[image_breed]
    
    #One hot encoder
    y = to_categorical(y)
    #shuffle    
    ind = np.random.permutation(data_size)
    X = X[ind]
    y = y[ind]
    print('Ouptut Data Size: ', X.shape)
    print('Ouptut Label Size: ', y.shape)
    return X, y

In [7]:
#img_size chosen to be 331 to suit the used architectures.
img_size = (331,331,3)
X, y = images_to_array(train_dir, labels_dataframe, img_size)

100%|██████████| 10222/10222 [01:01<00:00, 167.16it/s]


Ouptut Data Size:  (10222, 331, 331, 3)
Ouptut Label Size:  (10222, 120)


In [8]:
def get_features(model_name, data_preprocessor, input_size, data):
    base_model = model_name(weights='imagenet', include_top=False,
                            input_shape=input_size)

    model = Sequential()
    model.add(InputLayer(input_shape = input_size))
    model.add(Lambda(preprocess_input))
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    
    feature_maps = model.predict(data)
    
    print('Feature maps shape: ', feature_maps.shape)
    return feature_maps

In [9]:
# Extract features using InceptionV3 as extractor.
from keras.applications.inception_v3 import InceptionV3, preprocess_input
inception_preprocessor = preprocess_input
inception_features = get_features(InceptionV3,inception_preprocessor,img_size, X)

Feature maps shape:  (10222, 2048)


In [10]:
# Extract features using VGG16 as extractor.
from keras.applications.vgg16 import VGG16, preprocess_input
vgg16_preprocessor = preprocess_input
vgg16_features = get_features(VGG16,vgg16_preprocessor,img_size, X)

58892288/58889256 [==============================] - 1s 0us/step
Feature maps shape:  (10222, 512)


In [11]:
# Extract features using ResNet50 as extractor.
from keras.applications.resnet50 import ResNet50, preprocess_input
resnet_preprocessor = preprocess_input
resnet_features = get_features(ResNet50,resnet_preprocessor,img_size, X)

/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step
Feature maps shape:  (10222, 2048)


In [12]:
del X

In [13]:
from keras.callbacks import EarlyStopping
EarlyStop_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
my_callback=[EarlyStop_callback]

In [14]:
model = Sequential()
model.add(InputLayer(input_shape = (vgg16_features.shape[1:] )))
model.add(Dropout(0.7))
model.add(Dense(120, activation = 'softmax'))
    
model.compile(loss = 'categorical_crossentropy', optimizer ='Adam', metrics = ['accuracy'])
h=model.fit(vgg16_features, y,
            batch_size=128,
            epochs=60,
            validation_split=0.1,
            callbacks=my_callback)

Train on 9199 samples, validate on 1023 samples
Epoch 1/60
9199/9199 [==============================] - 0s 45us/step - loss: 22.2606 - accuracy: 0.0186 - val_loss: 6.1951 - val_accuracy: 0.1105
Epoch 2/60
9199/9199 [==============================] - 0s 44us/step - loss: 14.9255 - accuracy: 0.0592 - val_loss: 3.4658 - val_accuracy: 0.2893
Epoch 3/60
9199/9199 [==============================] - 0s 52us/step - loss: 10.6113 - accuracy: 0.1259 - val_loss: 2.4979 - val_accuracy: 0.4418
Epoch 4/60
9199/9199 [==============================] - 0s 45us/step - loss: 8.2150 - accuracy: 0.1821 - val_loss: 1.8953 - val_accuracy: 0.5396
Epoch 5/60
9199/9199 [==============================] - 0s 33us/step - loss: 6.4752 - accuracy: 0.2398 - val_loss: 1.6701 - val_accuracy: 0.5748
Epoch 6/60
9199/9199 [==============================] - 0s 34us/step - loss: 5.3893 - accuracy: 0.2917 - val_loss: 1.4895 - val_accuracy: 0.6002
Epoch 7/60
9199/9199 [==============================] - 0s 30us/step - loss: 4.

In [15]:
model = Sequential()
model.add(InputLayer(input_shape = (resnet_features.shape[1:] )))
model.add(Dropout(0.7))
model.add(Dense(120, activation = 'softmax'))
    
model.compile(loss = 'categorical_crossentropy', optimizer ='Adam', metrics = ['accuracy'])
h=model.fit(resnet_features, y,
            batch_size=128,
            epochs=60,
            validation_split=0.1,
            callbacks=my_callback)

Train on 9199 samples, validate on 1023 samples
Epoch 1/60
9199/9199 [==============================] - 0s 42us/step - loss: 4.0943 - accuracy: 0.1344 - val_loss: 2.1389 - val_accuracy: 0.5973
Epoch 2/60
9199/9199 [==============================] - 0s 30us/step - loss: 2.1481 - accuracy: 0.4390 - val_loss: 1.3216 - val_accuracy: 0.7234
Epoch 3/60
9199/9199 [==============================] - 0s 31us/step - loss: 1.5176 - accuracy: 0.6007 - val_loss: 1.0335 - val_accuracy: 0.7498
Epoch 4/60
9199/9199 [==============================] - 0s 31us/step - loss: 1.1988 - accuracy: 0.6747 - val_loss: 0.8798 - val_accuracy: 0.7693
Epoch 5/60
9199/9199 [==============================] - 0s 30us/step - loss: 1.0309 - accuracy: 0.7139 - val_loss: 0.7978 - val_accuracy: 0.7947
Epoch 6/60
9199/9199 [==============================] - 0s 30us/step - loss: 0.8832 - accuracy: 0.7521 - val_loss: 0.7313 - val_accuracy: 0.8016
Epoch 7/60
9199/9199 [==============================] - 0s 30us/step - loss: 0.786

In [16]:
model = Sequential()
model.add(InputLayer(input_shape = (inception_features.shape[1:] )))
model.add(Dropout(0.7))
model.add(Dense(120, activation = 'softmax'))
    
model.compile(loss = 'categorical_crossentropy', optimizer ='Adam', metrics = ['accuracy'])
h=model.fit(inception_features, y,
            batch_size=128,
            epochs=60,
            validation_split=0.1,
            callbacks=my_callback)

Train on 9199 samples, validate on 1023 samples
Epoch 1/60
9199/9199 [==============================] - 0s 42us/step - loss: 2.8423 - accuracy: 0.4745 - val_loss: 1.0319 - val_accuracy: 0.8759
Epoch 2/60
9199/9199 [==============================] - 0s 30us/step - loss: 0.7968 - accuracy: 0.8498 - val_loss: 0.4823 - val_accuracy: 0.9042
Epoch 3/60
9199/9199 [==============================] - 0s 30us/step - loss: 0.4946 - accuracy: 0.8846 - val_loss: 0.3779 - val_accuracy: 0.9062
Epoch 4/60
9199/9199 [==============================] - 0s 30us/step - loss: 0.3886 - accuracy: 0.9019 - val_loss: 0.3424 - val_accuracy: 0.9071
Epoch 5/60
9199/9199 [==============================] - 0s 30us/step - loss: 0.3367 - accuracy: 0.9085 - val_loss: 0.3116 - val_accuracy: 0.9198
Epoch 6/60
9199/9199 [==============================] - 0s 30us/step - loss: 0.3032 - accuracy: 0.9124 - val_loss: 0.3058 - val_accuracy: 0.9140
Epoch 7/60
9199/9199 [==============================] - 0s 45us/step - loss: 0.277

In [17]:
def images_to_array2(data_dir, labels_dataframe, img_size = (224,224,3)):
   
    images_names = labels_dataframe['id']
    data_size = len(images_names)
    X = np.zeros([data_size, img_size[0], img_size[1], 3], dtype=np.uint8)
    
    for i in tqdm(range(data_size)):
        image_name = images_names[i]
        img_dir = os.path.join(data_dir, image_name+'.jpg')
        img_pixels = tf.keras.preprocessing.image.load_img(img_dir, target_size=img_size)
        X[i] = img_pixels
        
    print('Ouptut Data Size: ', X.shape)
    return X

test_data = images_to_array2(test_dir, sample_df, img_size)

100%|██████████| 10357/10357 [00:48<00:00, 213.53it/s]

Ouptut Data Size:  (10357, 331, 331, 3)


In [ ]:
#Extract test data features.
inception_features = get_features(InceptionV3, inception_preprocessor, img_size, test_data)

In [ ]:
#Free up some space.
del test_data

In [ ]:
#Predict test labels given test data features.
y_pred = model.predict(inception_features, batch_size=128)

In [ ]:
#Create submission file
for b in dog_breeds:
    sample_df[b] = y_pred[:,class_to_num[b]]
sample_df.to_csv('pred.csv', index=None)